Testing of BioKotlin Ranges

In [1]:
//If this does not exist run from cmdline: ./gradlew shadowjar
@file:DependsOn("../build/libs/biokotlin-0.03-all.jar")


In [2]:
import biokotlin.genome.*
// import seq* as Ranges will use NucSeqRecord
import biokotlin.seq.NUC.*
import biokotlin.seq.*


In [3]:
val seq = Seq("GCAGAT")

In [4]:
val rec1 = NucSeqRecord(NucSeq("ATAACACAGAGATATATC"),"1")

In [6]:
// This creates a subset of the sequence in rec1 - just seq from positions 1 to 6
// Note the array positions are 0-based, so this should pull TAACAC
val subSettedSeq = rec1[1..6]

In [7]:
// create a Sequence Range (SRange) object
val sRange = rec1.range(8..12)

In [8]:
println(sRange)

1:8..1:12


In [10]:
val sRangeFlanked = sRange.flankBoth(5)

In [11]:
println(sRangeFlanked)

[1:3..1:7, 1:13..1:17]


In [12]:
val sRangeFlankRight = sRange.flankRight(5)

In [13]:
println(sRangeFlankRight)

1:13..1:17


In [14]:
val dnaString = "ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC"
val dnaString2 = "ACGTGGTGAATATATATGCGCGCGTGCGTGGACGTACGTACGTACGTATCAGTCAGCTGAC"
val record1 = NucSeqRecord(NucSeq(dnaString), "Sequence 1", description = "The first sequence",
                annotations = mapOf("key1" to "value1"))
val record2 = NucSeqRecord(NucSeq(dnaString2), "Sequence 2", description = "The second sequence",
                annotations = mapOf("key1" to "value1"))

var range1 = SeqPositionRanges.of(record1,8..28)
var range2 = SeqPositionRanges.of(record2,3..19)
var range3 = SeqPositionRanges.of(SeqPosition(record1, 32),SeqPosition(record1,40))
var range4 = record2.range(25..40)

// is there a way to default to this setOf without typing entire class?
// the setOf in genome.Ranges is a java Navigable set, which does the sorting
var setRanges = overlappingSetOf(SeqPositionRangeComparator.sprComparator,range1, range4, range3, range2)

In [15]:
for (range in setRanges) {
    println(range.toString())
    println()
}

Sequence 1:8..Sequence 1:28

Sequence 1:32..Sequence 1:40

Sequence 2:3..Sequence 2:19

Sequence 2:25..Sequence 2:40



In [16]:
// if a Java/Kotlin non-sorted set were used above, this would sort the set by seqRecord, then lowerEndpoint site
var setRangesSorted = setRanges.toSortedSet(SeqPositionRangeComparator.sprComparator)


In [17]:
for (range in setRangesSorted) {
    println(range.toString())
    println()
}

Sequence 1:8..Sequence 1:28

Sequence 1:32..Sequence 1:40

Sequence 2:3..Sequence 2:19

Sequence 2:25..Sequence 2:40



[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=8, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=28, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)]

[SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGACTAGCTGAC
, site=32, comparator=biokotlin.genome.SeqPositionAlphaComparator@55b84b18)..SeqPosition(seqRecord=ID: Sequence 1
Description: The first sequence
key1: value1
Sequence: ACGTGGTGAATATATATGCGCGCGTGCGTGGATCAGTCAGTCATGCATGCATGTGTGTACACACATGTGATCGTAGCTAGCTAGCTGACTGAC